In [55]:
import polars as pl
import time

In [ ]:
edges = pl.read_csv('monarch-kg/monarch-kg_edges.tsv', separator='\t')

In [ ]:
edges.head()

In [ ]:
d2p_edges = edges.filter(pl.col("category").str.contains('biolink:DiseaseToPhenotypicFeatureAssociation'))
d2p_filtered = d2p_edges.select(
    pl.col("subject"),
    pl.col("object")
)
print(d2p_filtered)


# disease_phenotypes = edges[(edges['category'] == 'biolink:DiseaseToPhenotypicFeatureAssociation') & (edges['subject'].str.contains("MONDO"))]

In [ ]:
phen_rel = pl.read_csv('phenio-relation-graph.tsv', separator='\t')

In [ ]:
phen_rel.columns = ['subject', 'predicate', 'object']

In [ ]:
phen_rel.head()

In [ ]:
pr_isa = phen_rel.filter(
    (pl.col("predicate") == 'rdfs:subClassOf') &
    (pl.col("subject").str.contains('HP:')) &
    (pl.col("object").str.contains('HP:'))
)
pr_isa

In [ ]:
merged_d2p = pl.join(left=d2p_filtered, right=pr_isa, how='left', left_on='object', right_on='subject')
merged_d2p.head()

In [ ]:
merged_d2p = d2p_filtered.join(pr_isa, left_on="object", right_on="subject", how="left")
merged_d2p.head()

In [237]:
# example disease MONDO:0030017
mondo_0030017 = ['HP:0000001',
 'HP:0000118',
 'HP:0000707',
 'HP:0000924',
 'HP:0001252',
 'HP:0001319',
 'HP:0001367',
 'HP:0001384',
 'HP:0001507',
 'HP:0001510',
 'HP:0001511',
 'HP:0001518',
 'HP:0001871',
 'HP:0001939',
 'HP:0001941',
 'HP:0002011',
 'HP:0002013',
 'HP:0002017',
 'HP:0002020',
 'HP:0002086',
 'HP:0002151',
 'HP:0002188',
 'HP:0002587',
 'HP:0002644',
 'HP:0002795',
 'HP:0002813',
 'HP:0002814',
 'HP:0002867',
 'HP:0003011',
 'HP:0003170',
 'HP:0003236',
 'HP:0003272',
 'HP:0003287',
 'HP:0003808',
 'HP:0004323',
 'HP:0004325',
 'HP:0004360',
 'HP:0004364',
 'HP:0005262',
 'HP:0008347',
 'HP:0008807',
 'HP:0008872',
 'HP:0008972',
 'HP:0010876',
 'HP:0011017',
 'HP:0011021',
 'HP:0011024',
 'HP:0011400',
 'HP:0011458',
 'HP:0011804',
 'HP:0011842',
 'HP:0011844',
 'HP:0011922',
 'HP:0011923',
 'HP:0011924',
 'HP:0011968',
 'HP:0012103',
 'HP:0012337',
 'HP:0012415',
 'HP:0012447',
 'HP:0012448',
 'HP:0012639',
 'HP:0012719',
 'HP:0025031',
 'HP:0025032',
 'HP:0025270',
 'HP:0025354',
 'HP:0032180',
 'HP:0033127',
 'HP:0040064',
 'HP:0040068',
 'HP:0040069',
 'HP:0040081',
 'HP:0100491',
 'HP:0500165']

In [294]:
import numpy as np
all_sets = []
def jaccard_index(set1, set2):
    set1 = set(set1.split(","))
    all_sets.append(set1)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union


expanded_diseases = merged_d2p.groupby("subject").agg(
    pl.col("object_right").str.concat(",")
)
start = time.time()
scored_diseases = expanded_diseases.with_columns(
      expanded_diseases['object_right'].apply(lambda x: jaccard_index(x, mondo_0030017)).alias('jaccard')
    )
end = time.time()
print(end-start)
scored_diseases.sort('jaccard', descending=True).head(10)

0.6336119174957275


subject,object_right,jaccard
str,str,f64
"""MONDO:0030017""","""HP:0000001,HP:…",1.0
"""MONDO:0009703""","""HP:0000001,HP:…",0.33945
"""MONDO:0032635""","""HP:0000001,HP:…",0.330357
"""MONDO:0012510""","""HP:0000001,HP:…",0.323741
"""MONDO:0032628""","""HP:0000001,HP:…",0.32
"""MONDO:0032629""","""HP:0000001,HP:…",0.318681
"""MONDO:0020727""","""HP:0000001,HP:…",0.316239
"""MONDO:0032627""","""HP:0000001,HP:…",0.303922
"""MONDO:0033282""","""HP:0000001,HP:…",0.303279
